**© 2021 DataRobot, Inc. All rights reserved.**

**Proprietary and Confidential**

# Bias Testing outside of DataRobot

### Overview

**Purpose:**

This notebook allows users to access an existing project, specifiy a particular model, then calculate bias and fairness scores using prediction and training data. The notebook allows for the prediction balance and favorable predictive value parity

<br>

**Usage:**

User must input values for the following variables.

1. *token* - api taken generated for this user in the developer settings

2. *projectId* - ID of the project from the list that you wish to use

3. *modelId* - ID of the model under the project you wish to use

4. *predictionId* (optional) - ID of training prediction dataset if it already exists

5. *datasetId* - ID of training data to be saved and read in locally

6. *path* - local path for the training data to be saved/read from

7. *pred_col* - prediction column

8. *target* - target column

9. *fav_outcome* - favorable outcome of the target. Also set fav_outcome_updated variable later with encoded version. 

10. *unfav_outcome* - unfavorable outcome of the target. Also set unfav_outcome_updated variable later with encoded version. 

11. *prot_att* - protected attribute column

<br>


**Assumptions:**

1. Project already exists
2. Project was created using dataset stored in AI Catalog
3. User has a token generated in the developer settings within the platform
4. Target column in Yes / No format
5. No missing values in **prot_att** field
6. Don't use model that is trained on full validation set since predictions won't be valid

<br>

### Import packages

In [1]:
import numpy as np
import pandas as pd
import datarobot as dr
import operator

# Set pandas options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

### Accessing Project via API

In [2]:
# Establish connection to DR client - specify your own API token and confirm environment endpoint
token = ''
dr.Client(token=token, endpoint="https://app.datarobot.com/api/v2")

In [3]:
# List all projects to find which one to use (only first 10)
for p in dr.Project.list()[:10]:
    print (p.id, p.project_name)

61280731357813f2c67af462 DR_Demo_LendingClub_Guardrails_Fairness (Pred Balance)
6126a1d72884cf9859a6ce6a datarobot_visible MCH (prod 8/24)
6124071faf94178cb9b5de7b DR_Demo_Bond_trading_RFQ.csv (B&F)
6123eb6236256f7f1bb5de47 10k_diabetes (MetaL query PID)
6123a5a372e167ace8b5e7b5 fast iron 100k data.csv
611fcc79f1f43bdd4ab5e068 HR Hiring (Bias & Fairness).csv
611fab3a7e86848e575fae3e External Predictions Regression.csv
611e758122e50925105fb206 Motorcycle_insurance_claims_leak.csv
611d1f5a1e695fa8aa16c16e 10k_diabetes.csv
611d0a315c929ac36516bf93 10k_diabetes (SHAP project)


In [4]:
# Specify your own projectId
projectId = '61280731357813f2c67af462'
project = dr.Project.get(projectId)

### Accessing Model via API

In [5]:
def retrieve_leaderboard(project, metric=project.metric):

    # Create an empty list to store the leaderboard to start
    leaderboard = []
    # Get all of the models trained so far as we did above
    models = project.get_models()
    # Iterate over each of the models extracting different
    # pieces of relevant information as we did above
    for model in models:
        # Store the results for a specific metric - can also use 'AUC', etc.
        temp = model.metrics[metric]
        # Store the name of the model
        temp["Model"] = str(model)
        # Store the ID of the model
        temp["model_id"] = model.id
        # Store the feature list used to create the model
        temp["featurelist"] = model.featurelist_name
        # Store what sample percentage (%) was used for training
        temp["sample_pct"] = model.sample_pct
        # Append this list to leaderboard and move to next model
        leaderboard.append(temp)

    # Store leaderboard list as a pandas DataFrame    
    leaderboard_df = pd.DataFrame(leaderboard)[
        [
            "Model",
            "model_id",
            "featurelist",
            "sample_pct",
            "validation",
            "crossValidation",
            "holdout"
        ]
    ]

    #Show first 10 models
    display(leaderboard_df.head(10))

    # Return this leaderboard to explore further
    return leaderboard_df

In [6]:
# Create pandas DataFrame which shows our leaderboard
leaderboard_df = retrieve_leaderboard(project)

,Model,model_id,featurelist,sample_pct,validation,crossValidation,holdout
0,Model('Light Gradient Boosted Trees Classifier...,612808a2905f1a621923e7af,Informative Features - Leakage Removed,64.0,0.37354,0.372864,NaN
1,Model('AVG Blender'),61280c743991e72a8d23e7de,Informative Features - Leakage Removed,64.0,0.37485,0.372398,NaN
2,Model('eXtreme Gradient Boosted Trees Classifi...,612807cebf62f27c3562925d,Informative Features - Leakage Removed,32.0,0.37492,NaN,NaN
3,Model('Light Gradient Boosted Trees Classifier...,61280b7b13ff4fabcb7926d4,Informative Features - Leakage Removed,80.0,0.37650,0.373530,NaN
4,Model('eXtreme Gradient Boosted Trees Classifi...,612808a2905f1a621923e7ac,Informative Features - Leakage Removed,64.0,0.37701,0.375070,NaN
5,Model('RandomForest Classifier (Gini)'),612808a2905f1a621923e7b2,Informative Features - Leakage Removed,64.0,0.37714,0.377238,NaN
6,Model('Light Gradient Boosted Trees Classifier...,61280bfe0edeed905acc2c2a,Informative Features - Leakage Removed,100.0,0.37793,0.374020,0.37643
7,Model('Light Gradient Boosted Trees Classifier...,61280b037cf40bf876e58e48,DR Reduced Features M19,64.0,0.37879,0.374530,NaN
8,Model('Light Gradient Boosted Trees Classifier...,612807d0bf62f27c35629266,Informative Features - Leakage Removed,32.0,0.37959,NaN,NaN
9,Model('RandomForest Classifier (Gini)'),612807cfbf62f27c35629262,Informative Features - Leakage Removed,32.0,0.37961,NaN,NaN


In [7]:
# Select model to use from leaderboard results
modelId = '612808a2905f1a621923e7af'
model = dr.Model.get(project=projectId, model_id=modelId)

### Get Training Predictions via API and Store as Dataframe

In [15]:
def get_train_preds(projectId, model):
    try:
        trainPredJob = model.request_training_predictions(dr.enums.DATA_SUBSET.ALL)
        trainPreds = trainPredJob.get_result_when_complete()
        trainPredsDf = trainPreds.get_all_as_dataframe()
        return trainPredsDf
    except Exception as e:
        print("Training predictions already exist. Execute the next two cells to retrieve existing predictions")


In [16]:
#Store training predictions as dataframe
preds = get_train_preds(projectId, model)

Training predictions already exist. Execute the next two cells to retrieve existing predictions


Skip next two cells if previous line executed successfully

In [17]:
# for i in dr.TrainingPredictions.list(projectId):
#     print(i.prediction_id, i.data_subset)

61280c761413a738696a87f8 validationAndHoldout
61281683663cf91d0149c41f all


In [18]:
# prediction_id = '61281683663cf91d0149c41f'
# preds = dr.TrainingPredictions.get(projectId, prediction_id).get_all_as_dataframe()

In [19]:
# Check that we have predictions
display(preds.head())
display(preds.shape)
display(preds[preds.row_id == 2930])

,row_id,partition_id,prediction,class_Yes,class_No
0,0,3.0,No,0.097471,0.902529
1,1,1.0,No,0.254543,0.745457
2,2,Holdout,No,0.082933,0.917067
3,3,Holdout,No,0.099547,0.900453
4,4,4.0,No,0.292337,0.707663


(10000, 5)

,row_id,partition_id,prediction,class_Yes,class_No
2930,2930,Holdout,Yes,0.524906,0.475094


In [20]:
preds["partition_id"].value_counts()

Holdout    2000
3.0        1600
1.0        1600
4.0        1600
2.0        1600
0.0        1600
Name: partition_id, dtype: int64

### Get Training Data as .csv then Read in as Dataframe

In [21]:
# List datasets associated with project and find which ID to use
for ds in dr.Dataset.list():
    if project.id in [p.id for p in ds.get_projects()]:
        print(ds)
        break

Dataset(name='DR_Demo_LendingClub_Guardrails_Fairness.csv', id='5fc798b431070f1b9acea498')


In [22]:
# Specify dataset id from above and create dataset object
dataset_id = '5fc798b431070f1b9acea498'
dataset = dr.Dataset.get(dataset_id)

# Save dataset locally then create DataFrame
path = './lending_club.csv'
dataset.get_file(path)
train_df = pd.read_csv(path)
display(train_df.head())
display(train_df.shape)

,is_bad,member_id,loan_status,emp_title,emp_length,home_ownership,annual_inc,pymnt_plan,desc,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,revol_util_percent,policy_code,gender,race
0,Yes,M1236516,Charged Off,Aqua Sun Lawn and Landscaping,5 years,RENT,50000.0,n,NaN,282xx,NC,7.99,1.0,12/1/1998,1.0,19.0,NaN,3.0,0.0,13.9,9.0,False,0.0,NaN,14%,1,Male,Black
1,No,M0978861,Fully Paid,USDA/FMMA,9 years,RENT,34200.0,n,NaN,605xx,IL,22.74,2.0,11/1/1971,2.0,14.0,45.0,10.0,1.0,80.6,31.0,False,0.0,NaN,81%,1,Female,White
2,No,M0767668,Fully Paid,Doublepark LLC,3 years,RENT,30000.0,n,NaN,331xx,FL,2.36,0.0,7/1/2000,0.0,72.0,NaN,5.0,0.0,11.6,8.0,False,0.0,NaN,12%,1,Female,White
3,No,M0879043,Fully Paid,City of Rochester,3 years,MORTGAGE,51000.0,n,NaN,146xx,NY,11.20,0.0,5/1/1998,1.0,NaN,NaN,13.0,0.0,0.0,23.0,False,0.0,NaN,0%,1,Female,White
4,No,M0661671,Fully Paid,Roger George Rentals,9 years,RENT,44004.0,n,NaN,916xx,CA,22.47,0.0,5/1/2002,1.0,NaN,NaN,16.0,0.0,94.5,32.0,False,0.0,NaN,94%,1,Female,White


(10000, 28)

### Join Predictions with Training Data

In [23]:
# Create training dataframe with only pertinent columns - replace with your own 
cols = ['is_bad', 'gender']
train_reduced_df = train_df[cols]

In [24]:
# Join training data with preds on row_id in preds to index in training
preds_and_train_df = pd.merge(left=train_reduced_df, right=preds, left_index=True, right_on="row_id")
display(preds_and_train_df.shape)

(10000, 7)

In [25]:
# Setup variables for columns/outcomes for easier use
pred_col = 'prediction'
target = 'is_bad'
fav_outcome = 0 #Use 0 or 1 for 'No', 'Yes' (i.e. Boolean representation)
unfav_outcome = 1 #'Use 0 or 1 for 'No', 'Yes' (i.e. Boolean representation)
fav_outcome_col = 'class_No' #Use raw target label
unfav_outcome_col = 'class_Yes' #Use raw target label
prot_att = 'gender'

### Adjust Prediction Threshold

In [26]:
# Find the threshold that maximizes F1 score
roc = model.get_roc_curve('validation')
pred_threshold = roc.get_best_f1_threshold()
print(pred_threshold)

0.15222608713304683


In [27]:
# Update predictions based on threshold
preds_and_train_thresh_df = preds_and_train_df.copy()
preds_and_train_thresh_df[pred_col] = np.where((preds_and_train_thresh_df.class_Yes > pred_threshold), 'Yes', 'No')
display(preds_and_train_thresh_df.head())

,is_bad,gender,row_id,partition_id,prediction,class_Yes,class_No
0,Yes,Male,0,3.0,No,0.097471,0.902529
1,No,Female,1,1.0,Yes,0.254543,0.745457
2,No,Female,2,Holdout,No,0.082933,0.917067
3,No,Female,3,Holdout,No,0.099547,0.900453
4,No,Female,4,4.0,Yes,0.292337,0.707663


### Favorable / Unfavorable Class Balance

In [28]:
# Create aggregated df for favorable / unfavorable class balance
preds_balance_df = preds_and_train_thresh_df.copy()
# TODO: Shoop, I think preds_balance_df needs to be
# subset down to ONLY the Validation partition (i.e. partition == 0.0)
preds_balance_df = preds_balance_df[preds_balance_df["partition_id"] == "0.0"]

preds_balance_df[target] = np.where(preds_balance_df[target] == 'Yes', 1, 0)
preds_balance_df[pred_col] = np.where(preds_balance_df[pred_col] == 'Yes', 1, 0)
preds_balance_df = preds_balance_df.groupby([prot_att, target])[pred_col].mean().reset_index(name='mean_predicted_prob')

In [29]:
def favClassBalance(df, fairnessThreshold, favOutcome, protectedAtt):
    protAttClasses = list(df[protectedAtt].unique())
    favBalResults = {c: 0 for c in protAttClasses}

    FavMeanPredictedProb = df[df[target] == favOutcome].set_index(protectedAtt)['mean_predicted_prob']
    favBalResults.update(FavMeanPredictedProb.to_dict())
    favBalResultsSorted = dict(sorted(favBalResults.items(), key=operator.itemgetter(1)))#,reverse=True))

    if favOutcome == 0:
        for k in favBalResultsSorted:
            favBalResultsSorted[k] = 1-(favBalResultsSorted[k])

    favPrivFairnessVal = list(favBalResultsSorted.items())[0][1]
    favPrivFairnessKey = list(favBalResultsSorted.items())[0][0]

    relativeResults = dict(favBalResultsSorted)

    for k in relativeResults:
        if k == favPrivFairnessKey:
            relativeResults[k] = 1

        else:
            relativeResults[k] = round(relativeResults[k] / favPrivFairnessVal, 2)

    
    
    print('Absolute Fairness For Favorable Class Balance:')
    for k, v in favBalResultsSorted.items():
        print(k, v)

    print('\nRelative Fairness for Favorable Class Balance:')
    for k, v in relativeResults.items():
        print(k, v)

    biasedResults = dict((k, v) for k, v in relativeResults.items() if v < fairnessThreshold)

    print('\nBiased Results:')
    for k, v in biasedResults.items():
        print(k, v)

In [30]:
favClassBalance(preds_balance_df, 0.8, fav_outcome, prot_att)

Absolute Fairness For Favorable Class Balance:
Male 0.8994169096209913
Female 0.548158640226629

Relative Fairness for Favorable Class Balance:
Male 1
Female 0.61

Biased Results:
Female 0.61


In [31]:
# TODO: hmmmmmmmmmm, Absolute fairness scores, Male is close, but Female is off :confused-face:

In [32]:
def unfavClassBalance(df, fairnessThreshold, unfavOutcome, protectedAtt):
    protAttClasses = list(df[protectedAtt].unique())
    unfavBalResults = {c: 0 for c in protAttClasses}

    unfavMeanPredictedProb = df[df[target] == unfavOutcome].set_index(protectedAtt)['mean_predicted_prob']
    unfavBalResults.update(unfavMeanPredictedProb.to_dict())
    unfavBalResultsSorted = dict(sorted(unfavBalResults.items(), key=operator.itemgetter(1)))#,reverse=True))

    if unfavOutcome == 1:
        for k in unfavBalResultsSorted:
            unfavBalResultsSorted[k] = 1-(unfavBalResultsSorted[k])

    favPrivFairnessVal = list(unfavBalResultsSorted.items())[0][1]
    favPrivFairnessKey = list(unfavBalResultsSorted.items())[0][0]

    relativeResults = dict(unfavBalResultsSorted)

    for k in relativeResults:
        if k == favPrivFairnessKey:
            relativeResults[k] = 1

        else:
            relativeResults[k] = round(relativeResults[k] / favPrivFairnessVal, 2)

    
    
    print('Absolute Fairness For Favorable Class Balance:')
    for k, v in unfavBalResultsSorted.items():
        print(k, v)

    print('\nRelative Fairness for Favorable Class Balance:')
    for k, v in relativeResults.items():
        print(k, v)

    biasedResults = dict((k, v) for k, v in relativeResults.items() if v < fairnessThreshold)

    print('\nBiased Results:')
    for k, v in biasedResults.items():
        print(k, v)

In [33]:
unfavClassBalance(preds_balance_df, 0.8, unfav_outcome, prot_att)

Absolute Fairness For Favorable Class Balance:
Male 0.7857142857142857
Female 0.3709677419354839

Relative Fairness for Favorable Class Balance:
Male 1
Female 0.47

Biased Results:
Female 0.47


### Favorable Predictive Value Parity

In [38]:
fav_pred_value_df = preds_and_train_thresh_df.copy()
fav_pred_value_df = fav_pred_value_df[fav_pred_value_df["partition_id"] == "0.0"]

In [39]:
fav_pred_value_df.groupby([prot_att, pred_col])[target].aggregate(["count"])

count
gender prediction       
Female No            433
       Yes           397
Male   No            683
       Yes            87

In [40]:
# Helper method function for aggregate "true_predictions" (i.e. correct) predictions
def true_predictions_count(actual_prediction_column):
            return actual_prediction_column[fav_pred_value_df[pred_col] == fav_pred_value_df[target]].count()

In [41]:
fav_pred_value_df_agg = fav_pred_value_df.groupby([prot_att, pred_col])[target].aggregate(["count", true_predictions_count]) \
            .rename(
                columns={
                    "count": "predictions_count",
                    true_predictions_count.__name__: "true_predictions_count",
                }
            ) \
            .reset_index()

# TODO: hack, change pred_col values from string (Yes/No) to boolean (1/0)
fav_pred_value_df_agg = fav_pred_value_df_agg.replace({"prediction": {"Yes": 1, "No": 0}})

display(fav_pred_value_df_agg)

,gender,prediction,predictions_count,true_predictions_count
0,Female,0,433,387
1,Female,1,397,78
2,Male,0,683,617
3,Male,1,87,18


In [42]:
def _count_predictions(aggregated_data):
    return aggregated_data.groupby(prot_att)["predictions_count"].sum()

In [43]:
def _count_favorable_predictions(aggregated_data):
    favorable_predictions = aggregated_data[
        aggregated_data[pred_col] == fav_outcome
    ]
    return _count_predictions(favorable_predictions)

In [44]:
def _count_true_predictions(aggregated_data):
    return aggregated_data.groupby(prot_att)["true_predictions_count"].sum()

In [45]:
def _count_true_favorable_predictions(aggregated_data):
    favorable_aggregated_data = aggregated_data[
        aggregated_data[pred_col] == fav_outcome
    ]
    return _count_true_predictions(favorable_aggregated_data)

In [46]:
def _count_false_favorable_predictions(aggregated_data):
    favorable_predictions = _count_favorable_predictions(aggregated_data)
    true_favorable_predictions = _count_true_favorable_predictions(aggregated_data)
    return favorable_predictions - true_favorable_predictions

In [47]:
def favorable_predictive_value(df, aggregated_data, protected_att):
    protected_att_classes = list(df[protected_att].unique())
    results = {c: 0 for c in protected_att_classes}
    true_favorable = _count_true_favorable_predictions(aggregated_data)
    false_favorable = _count_false_favorable_predictions(aggregated_data)
    scores = true_favorable / (true_favorable + false_favorable)
    results.update(scores.fillna(0).to_dict())
    return results

In [48]:
favorable_predictive_value(fav_pred_value_df, fav_pred_value_df_agg, prot_att)

{'Female': 0.8937644341801386, 'Male': 0.9033674963396779}